In [1]:
import torch
import torch.optim as optim
from tqdm import tqdm

from utils.encoder import encode_data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from utils.dataprovider import DataProvider
from torch.utils.data import DataLoader

import logging
import time
from utils.model import *
import os

In [2]:
config = {
    "model_name": "DeepNeo",
    "do_train": True,
    "do_test": True,
    "Data": {
        "epi_path": "dataset/mhc1.trainset.tsv",
        "hla_path": "dataset/HLAseq.dat",
        "max_len_hla": 369,
        "max_len_pep": 9,
        "val_size": 0.2,
    },
    "Train": {
        "checkpoint_name": "base_model0",
        "batch_size": 1024,
        "num_epochs": 100,
        "patience": 10,
        "learning_rate": 0.01,
        "log_file": "log.txt"
    }
}

In [3]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, log_file, num_epochs, patience, model_name):
    model.train()
    best_loss = float('inf')
    epochs_no_improve = 0
    early_stop = False
    logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - [Training process]: [base_model0]-[model0]-[Epoch %(epoch)04d] - time: %(time)3d s, train_acc: %(train_acc).5f, val_acc: %(val_acc).5f, train_loss: %(train_loss).5f, val_loss: %(val_loss).5f')

    for epoch in range(num_epochs):
        if early_stop:
            print(f'Early stopping at epoch {epoch}')
            break

        start_time = time.time()
        epoch_loss = 0
        val_loss = 0
        correct_train = 0
        total_train = 0
        correct_val = 0
        total_val = 0

        model.train()
        with tqdm(train_loader, unit="batch") as tepoch:
            for batch in tepoch:
                tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")

                batch_matrices, affinities = batch
                batch_matrices = batch_matrices.to(device)
                affinities = affinities.to(device)

                optimizer.zero_grad()
                outputs = model(batch_matrices)
                loss = criterion(outputs, affinities)
                
                # Regularization
                if config['model_name'] == 'DeepNeo':
                    l1_lambda = 0.0001
                    l2_lambda = 0.001
                    l2_reg = torch.tensor(0.).to(device)
                    for param in model.parameters():
                        l2_reg += torch.norm(param, 2)
                    loss += l2_lambda*l2_reg + l1_lambda*torch.norm(model.fc1.weight, 1)  # L1 norm of the fully connected layer
                
                loss.backward()
                optimizer.step()

                tepoch.set_postfix(loss=loss.item())
                epoch_loss += loss.item()

                # Accuracy 계산
                predicted = (outputs > 0.5).float()
                correct_train += (predicted == affinities).sum().item()
                total_train += affinities.size(0)

        avg_train_loss = epoch_loss / len(train_loader)
        train_acc = correct_train / total_train

        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                batch_matrices, affinities = batch
                batch_matrices = batch_matrices.to(device)
                affinities = affinities.to(device)
                
                outputs = model(batch_matrices)
                loss = criterion(outputs, affinities)
                val_loss += loss.item()

                # Accuracy 계산
                predicted = (outputs > 0.5).float()
                correct_val += (predicted == affinities).sum().item()
                total_val += affinities.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_acc = correct_val / total_val

        epoch_time = int(time.time() - start_time)
        logging.info('', extra={'epoch': epoch, 'time': epoch_time, 'train_acc': train_acc, 'val_acc': val_acc, 'train_loss': avg_train_loss, 'val_loss': avg_val_loss})

        # Early stopping and best model saving
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            if not os.path.exists('models'):
                os.makedirs('models')
            torch.save(model.state_dict(),f"models/{model_name}-best.pt")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                early_stop = True

In [4]:
# DataProvider 객체 생성
data_provider = DataProvider(
    epi_path=config["Data"]["epi_path"],
    hla_path=config["Data"]["hla_path"],
)
x, y = encode_data(data_provider)

# 데이터를 학습용과 검증용으로 나누기 (클래스 비율 유지)
x_train, x_val, y_train, y_val = train_test_split(
    x, y, stratify=y,
    test_size=config["Data"]["val_size"],
    random_state=42
    )

# numpy 배열을 텐서로 변환
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

# TensorDataset으로 변환
train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(x_val_tensor, y_val_tensor)
print(f"Samples in train set: {len(train_dataset)}")
print(f"Samples in validation set: {len(val_dataset)}")

# DataLoader를 사용하여 데이터를 배치로 로드
batch_size = config["Train"]["batch_size"]
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

Number of HLA alleles: 7044
Number of samples: 238429
Samples in train set: 190743
Samples in validation set: 47686


In [5]:
# 모델 초기화
model = model_map[config["model_name"]]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f'Model loaded on {device}')

# 손실 함수 및 옵티마이저 정의
criterion = nn.BCELoss()
optimizer = optim.SGD(
    model.parameters(),
    lr=config["Train"]["learning_rate"],
    momentum=0.9,
    )

# 모델 학습
train_model(
    model           = model,
    train_loader    = train_loader,
    val_loader      = val_loader,
    criterion       = criterion,
    optimizer       = optimizer,
    device          = device,
    log_file        = config["Train"]["log_file"],
    num_epochs      = config["Train"]["num_epochs"],
    patience        = config["Train"]["patience"],
    model_name      = config["Train"]["checkpoint_name"]
    )

Model loaded on cuda


Epoch 1/100:  13%|█▎        | 25/187 [00:03<00:18,  8.95batch/s, loss=0.561]